<p align="center"><img src="../Additionals/Empa-Workshops-Template-Banner2.png" alt="University Workshops" style="display: block; margin: 0 auto" height=/></p>

## Accelerator Workshops'a Hoşgeldiniz!

Empa Electronics tarafından düzenlenen Accelerators Workshops etkinlikleri serimizin Uçta Yapay Zeka adımına hoşgeldiniz. Bu açık-kaynak repository, workshop etkiliğimizde deneyimleyeceğiniz "ST Platformları İçin Edge-AI Çözümleri Geliştirme" uygulaması çalışma ortamını edinebilmeniz ve aktivitelere kolaylıkla eşlik edebilmeniz için sizinle paylaşılmıştır.

Bu script, uygulama içeriğinde geliştirilen modelin uç birimde testi için oluşturulmuştur.

**Uygulama Adımları:**

1. Gereksinimlerin Dahil Edilmesi
2. Seri Port ile Veri Örneği Okuma & Eğitilmiş Modeli Kullanarak Tahminleme

## 1. Gereksinimlerin Dahil Edilmesi

Gerekli modeller için import işlemi:

In [ ]:
import cv2
import numpy as np
import serial
from PIL import Image

2024-06-06 15:52:26.114092: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 15:52:26.148306: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 15:52:26.148606: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 15:52:26.806066: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## 3. Seri Port ile Veri Örneği Okuma & Eğitilmiş Modeli Kullanarak Tahminleme

Seri port konfigürasyonu için gerekli değerlerin tanımlanması:

In [ ]:
serial_port = '/dev/ttyACM0'   # Ubuntu
# serial_port = 'COM0'         # Windows
baud_rate = 115200
time_out = 2

Seri port üzerinden veri okuma & okunan veri örneklerinin tahminlenmesi

In [ ]:
class HandCharApp:
    def __init__(self, serialport, baudrate, timeout):
        self.serialport = serialport
        self.baudrate = baudrate
        self.timeout = timeout
        self.line = None
        self.name_class = None
        self.conf_vertical = -1
        self.conf_horizontal = -1
        self.conf_stanby = -1
        self.conf_circle = -1
        self.conf_triangle = -1
        
        self.read_background()
        self.name_app = "Hand Character Recognition - GUI - Empa Electronics"

    def read_data(self):
        with serial.Serial(self.serialport, self.baudrate, timeout=5) as ser_read:
            line = ser_read.readline()
            self.line = str(line).replace("b'", "").replace("\\n'", "").split(" ")
            
            if len(self.line) == 6 and self.line[0] in ["circle", "horizontal", "standby", "triangle", "vertical"]:
                self.name_class,  self.conf_circle, self.conf_horizontal, self.conf_stanby, self.conf_triangle, self.conf_vertical = self.line
            else:
                self.line = None

    def read_background(self):
        self.image_bg = cv2.cvtColor(np.array(Image.open("Additionals/Demo-Image-Hand-Character.png")), cv2.COLOR_RGB2BGR)

    def get_class_roi(self):
        image = self.image_bg
        self.roi_circle = image[:, :300].copy()
        self.roi_horizontal = image[:, 300:620].copy()
        self.roi_standby = image[:, 620:950].copy()
        self.roi_triangle = image[:, 950:1300].copy()
        self.roi_vertical = image[:, 1300:].copy()

    def paint_class_roi(self):
        
        if self.name_class == "circle":
            roi = self.roi_circle.copy()
            mask = np.all(roi == [255, 255, 255], axis=-1)
            roi[mask] = [0, 255, 255]
            self.image_gui[:, :300] = roi

        elif self.name_class == "horizontal":
            roi = self.roi_horizontal.copy()
            mask = np.all(roi == [255, 255, 255], axis=-1)
            roi[mask] = [0, 255, 255]
            self.image_gui[:, 300:620] = roi

        elif self.name_class == "standby":
            roi = self.roi_standby.copy()
            # print(self.roi_standby.shape, self.roi_standby.dtype)
            # print(self.roi_standby)
            mask = np.all(roi == [255, 255, 255], axis=-1)
            roi[mask] = [0, 255, 255]
            self.image_gui[:, 620:950] = roi

        elif self.name_class == "triangle":
            roi = self.roi_triangle.copy()
            mask = np.all(roi == [255, 255, 255], axis=-1)
            roi[mask] = [0, 255, 255]
            self.image_gui[:, 950:1300] = roi

        elif self.name_class == "vertical":
            roi = self.roi_vertical.copy()
            mask = np.all(roi == [255, 255, 255], axis=-1)
            roi[mask] = [0, 255, 255]
            self.image_gui[:, 1300:] = roi


    def print_confidences(self):
        self.image_gui = self.image_bg.copy()
        
        # print(self.conf_circle, self.conf_horizontal, self.conf_stanby, self.conf_triangle, self.conf_vertical)

        self.conf_circle = float(self.conf_circle)
        self.conf_horizontal = float(self.conf_horizontal)
        self.conf_stanby = float(self.conf_stanby)
        self.conf_triangle = float(self.conf_triangle)
        self.conf_vertical = float(self.conf_vertical)
           
        cv2.putText(self.roi_circle, f"{self.conf_circle*100}%", (130, 750), cv2.FONT_HERSHEY_DUPLEX, 1.3, (255, 255, 255), 2)
        cv2.putText(self.roi_horizontal, f"{self.conf_horizontal*100}%", (100, 750), cv2.FONT_HERSHEY_DUPLEX, 1.3, (255, 255, 255), 2)
        cv2.putText(self.roi_standby, f"{self.conf_stanby*100}%", (100, 750), cv2.FONT_HERSHEY_DUPLEX, 1.3, (255, 255, 255), 2)
        cv2.putText(self.roi_triangle, f"{self.conf_triangle*100}%", (100, 750), cv2.FONT_HERSHEY_DUPLEX, 1.3, (255, 255, 255), 2)
        cv2.putText(self.roi_vertical, f"{self.conf_vertical*100}%", (50, 750), cv2.FONT_HERSHEY_DUPLEX, 1.3, (255, 255, 255), 2)

    def display_gui(self):
        cv2.namedWindow(self.name_app, cv2.WINDOW_FREERATIO)
        cv2.imshow(self.name_app, self.image_gui)
        
    def print_to_terminal(self):
        print(self.line)
        
    def run_app(self):
        while True:
            self.read_data()
            # self.print_to_terminal()
            self.get_class_roi()
            self.print_confidences()
            self.paint_class_roi()
            self.display_gui()

            if cv2.waitKey(20) & 0xFF == ord('q'):
                break

        cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    app = HandCharApp(serial_port, baud_rate, time_out)
    app.run_app()